In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt

In [0]:
hog_extractor = cv2.HOGDescriptor((64, 64), (16, 16), (8, 8), (8, 8), 9)
hog_win_stride = (8, 8)
SVC = LinearSVC()
# SVC = SVC(gamma='scale')

In [0]:
class SVM_Data_Loader:
    def __init__(self, data_path, label, hog_extractor, hog_win_stride):
        X_list = []
        Y_list = []
        for image_name in os.listdir(data_path):
            try:
                file_name = os.path.join(data_path, image_name)
                image = cv2.resize(cv2.imread(file_name), (64, 128), interpolation=cv2.INTER_AREA)
                hog_feature_vector = hog_extractor.compute(image, hog_win_stride).T[0]
                X_list.append(hog_feature_vector)
                Y_list.append(label)
            except:
                continue

        self.X = np.asarray(X_list).astype(np.float64)
        self.Y = Y_list

    def get_X_and_Y(self):
        return self.X, self.Y

In [0]:
class SVM:
    def __init__(self, positive_data_loader, negative_data_loader, SVC):
        positive_X, positive_Y = positive_data_loader.get_X_and_Y()
        negative_X, negative_Y = negative_data_loader.get_X_and_Y()
        stack_X = np.vstack((positive_X, negative_X))
        stack_Y = np.asarray(positive_Y + negative_Y)

        self.X = StandardScaler().fit(stack_X).transform(stack_X)
        self.Y = stack_Y
        self.SVC = SVC

    def train(self):
        train_X, test_X, train_Y, test_Y = train_test_split(self.X, self.Y, test_size=0.2)
        self.SVC.fit(train_X, train_Y)
        score = self.SVC.score(test_X, test_Y)
        print("Accuracy:" + str(score * 100.0) + "%")

    def predict(self, test_X):
        return self.SVC.predict(test_X)

In [0]:
positive_data_loader = SVM_Data_Loader("/content/drive/My Drive/CSC420Project/dog_pos", 1, hog_extractor, hog_win_stride)

In [0]:
negative_data_loader = SVM_Data_Loader("/content/drive/My Drive/CSC420Project/dog_neg", 0, hog_extractor, hog_win_stride)

In [0]:
dog_svm = SVM(positive_data_loader, negative_data_loader, SVC)

In [0]:
dog_svm.train()

In [0]:
class Dog_Patch_Extractor:
    def __init__(self, SVM, hog_extractor, hog_win_stride):
        self.SVM = SVM
        self.hog_extractor = hog_extractor
        self.hog_win_stride = hog_win_stride
        self.cache = {}

    def extract_dog_patch_from_images(self, data_path, output_path):
        self.cache = {}
        for image_name in os.listdir(data_path):
            file_name = os.path.join(data_path, image_name)
            image = cv2.imread(file_name)
            window_size_list = [(int(image.shape[0] * 0.5), int(image.shape[1] * 0.5)), (int(image.shape[0] * 0.25), int(image.shape[1] * 0.25)), (int(image.shape[0] * 0.1), int(image.shape[1] * 0.1))]
            stride_size_list = [(int(window_size_list[0][0] * 0.1), int(window_size_list[0][1] * 0.1)), (int(window_size_list[1][0] * 0.1), int(window_size_list[1][1] * 0.1)), (int(window_size_list[2][0] * 0.1), int(window_size_list[2][1] * 0.1))]

            for i in range(3):
                window_size = window_size_list[i]
                stride_size = stride_size_list[i]
                found, possible_patches_row_col = self.find_possible_dog_patches_with_window(image, window_size, stride_size)
                if found:
                    final_patch_file_name, rectangle_file_name = self.extract_final_dog_patch(output_path, image_name, image, possible_patches_row_col)
                    if final_patch_file_name != "" and rectangle_file_name != "":
                        self.cache[file_name] = [final_patch_file_name, rectangle_file_name]
                        break



    def find_possible_dog_patches_with_window(self, image, window_size, stride_size):
        found = False
        possible_patches_row_col = []
        for col_end in range(window_size[0], image.shape[1], stride_size[0]):
            for row_end in range(window_size[1], image.shape[0], stride_size[1]):
                col_start = col_end - window_size[0]
                row_start = row_end - window_size[1]
                current_patch = image[row_start:row_end, col_start:col_end, :]
                hog_feature_vector = self.hog_extractor.compute(cv2.resize(current_patch, (64, 128), interpolation=cv2.INTER_AREA), self.hog_win_stride).T[0]
                if self.SVM.predict([hog_feature_vector]) == [1]:
                    possible_patches_row_col.append((row_start, row_end, col_start, col_end))
                    found = True
        return found, possible_patches_row_col



    def extract_final_dog_patch(self, output_path, image_name, image, possible_patches_row_col):
        row_start_list, row_end_list, col_start_list, col_end_list = [], [], [], []
        final_patch_file_name, final_rectangle_file_name = "", ""
        image_copy = image.copy()

        for patch_row_col in possible_patches_row_col:
            row_start_list.append(patch_row_col[0])
            row_end_list.append(patch_row_col[1])
            col_start_list.append(patch_row_col[2])
            col_end_list.append(patch_row_col[3])
            cv2.rectangle(image_copy, (patch_row_col[2], patch_row_col[0]), (patch_row_col[3], patch_row_col[1]), (0, 255, 0), thickness=2)
        if len(row_start_list) > 0 and len(row_end_list) > 0 and len(col_start_list) > 0 and len(col_end_list) > 0:
            final_row_start = min(row_start_list)
            final_row_end = max(row_end_list)
            final_col_start = min(col_start_list)
            final_col_end = max(col_end_list)

            final_patch_file_name = output_path + "/" + image_name + "_final_patch.jpg"
            print(cv2.imwrite(final_patch_file_name, image[final_row_start:final_row_end, final_col_start:final_col_end, :]))

            final_rectangle_file_name = output_path + "/" + image_name + "_rectangle.jpg"
            cv2.rectangle(image_copy, (final_col_start, final_row_start), (final_col_end, final_row_end), (255, 255, 0), thickness=3)
            print(cv2.imwrite(final_rectangle_file_name, image_copy))
            print("**********************************************************")

        return final_patch_file_name, final_rectangle_file_name



    def get_cache():
        return self.cache

In [0]:
dog_patch_extractor = Dog_Patch_Extractor(dog_svm, hog_extractor, hog_win_stride)

In [0]:
dog_patch_extractor.extract_dog_patch_from_images("/content/drive/My Drive/CSC420Project/test_svm_dog", "/content/drive/My Drive/CSC420Project/output2")